In [1]:
import tensorflow.keras as keras
import numpy as np

# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

4422102/4422102 [==============================] - 0s 0us/step


In [2]:
# Change the orientation of the dataset inorder to work

In [3]:
import numpy as np
from scipy.signal import correlate2d, convolve2d

class Convolution:
    
    def __init__(self, input_shape, filter_size, num_filters):
        input_height, input_width = input_shape
        self.num_filters = num_filters
        self.input_shape = input_shape
        
        # Size of outputs and filters or kernels or weigths
        
        self.filter_shape = (num_filters, filter_size, filter_size) # (3,3)
        self.output_shape = (num_filters, input_height - filter_size + 1, input_width - filter_size + 1)
        
        self.filters = np.random.randn(*self.filter_shape)
        self.biases = np.random.randn(*self.output_shape)
        
        
    def cross_correlation2D(self, input_mat, filter_mat):
        # Get the dimensions of the image and kernel
        image_height, image_width = input_mat.shape
        # (3, 3) shape
        kernel_height, kernel_width = filter_mat.shape

        # Calculate the output dimensions
        output_height = image_height - kernel_height + 1
        output_width = image_width - kernel_width + 1

        # Create an empty array to store the cross-correlation result
        result = np.zeros((output_height, output_width))

        # Perform cross-correlation
        for i in range(output_height):
            for j in range(output_width):
                patch = input_mat[i:i + kernel_height, j:j + kernel_width]
                result[i, j] = np.sum(patch * filter_mat)
        return result
        
        
    def forward(self, input_data):
        self.input_data = input_data
        # Initialized the input value
        output = np.zeros(self.output_shape)
        for i in range(self.num_filters):
            output[i] = correlate2d(self.input_data, self.filters[i], mode="valid")
        #Applying Relu Activtion function
        output = np.maximum(output, 0)
        
        return output 
    
    
    def backward(self, dL_dout, lr):
        # Create a random dL_dout array
        dL_dinput = np.zeros_like(self.input_data)
        dL_dfilters = np.zeros_like(self.filters)
        
        for i in range(self.num_filters):
                # Calculating the gradient of the filters or weights
                dL_dfilters[i] = correlate2d(self.input_data, dL_dout[i],mode="valid")
                
                # Calculating the gradient of the inputs
                dL_dinput += convolve2d(dL_dout[i], self.filters[i], mode="full")
                
        self.filters -= lr * dL_dfilters
        self.biases -= lr * dL_dout
    
        
        return dL_dinput


class MaxPool:
    
    def __init__(self, pool_size):
        self.pool_size = pool_size
        
    def forward(self, input_data):
        self.input_data = input_data
        self.num_channels, self.input_height, self.input_width = input_data.shape
        self.output_height = self.input_height // self.pool_size
        self.output_width = self.input_width // self.pool_size
        
        self.output = np.zeros((self.num_channels, self.output_height, self.output_width))
        

        for c in range(self.num_channels):
            for i in range(self.output_height):
                for j in range(self.output_width):
                    start_i = i * self.pool_size
                    start_j = j * self.pool_size

                    end_i = start_i + self.pool_size
                    end_j = start_j + self.pool_size 
                    patch = input_data[c, start_i:end_i, start_j:end_j]

                    self.output[c, i, j] = np.max(patch)
                        
        return self.output
    
    def backward(self, dL_dout, lr):
        dL_dinput = np.zeros_like(self.input_data)
        
        for c in range(self.num_channels):
            for i in range(self.output_height):
                for j in range(self.output_width):
                    start_i = i * self.pool_size
                    start_j = j * self.pool_size

                    end_i = start_i + self.pool_size
                    end_j = start_j + self.pool_size 
                    patch = self.input_data[c, start_i:end_i, start_j:end_j]
                    
                    mask = patch == np.max(patch)
                    
                    dL_dinput[c,start_i:end_i, start_j:end_j] = dL_dout[c, i, j] * mask
                    
        return dL_dinput
                        
class Fully_Connected:
    
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        #self.ouptut_neurons = output_neurons
        self.weights = np.random.randn(output_size, self.input_size)
        #print(self.weights)
        self.biases = np.random.rand(output_size, 1)
        
    
    def softmax(self, z):
        # Shift the input values to avoid numerical instability
        shifted_z = z - np.max(z)
        # Compute the exponentiated values
        exp_values = np.exp(shifted_z)

        # Compute the sum of exponentiated values using log-sum-exp trick
        sum_exp_values = np.sum(exp_values, axis=0)
        log_sum_exp = np.log(sum_exp_values)

        # Compute the softmax probabilities
        probabilities = exp_values / sum_exp_values

        return probabilities


    def softmax_derivative(self, s):
        return np.diagflat(s) - np.dot(s, s.T)
        
    def forward(self, input_data):
        self.input_data = input_data
        flattened_input = input_data.flatten().reshape(1, -1)
        self.z = np.dot(self.weights, flattened_input.T) + self.biases
        self.output = self.softmax(self.z)          
        return self.output
    
    def backward(self, dL_dout, lr):
        # Calculate the gradient of the loss with respect to the pre-activation (z)
        dL_dy = np.dot(self.softmax_derivative(self.output), dL_dout)
        # Calculate the gradient of the loss with respect to the weights (dw)
        dL_dw = np.dot(dL_dy, self.input_data.flatten().reshape(1, -1))

        # Calculate the gradient of the loss with respect to the biases (db)
        dL_db = dL_dy

        # Calculate the gradient of the loss with respect to the input data (dL_dinput)
        dL_dinput = np.dot(self.weights.T, dL_dy)
        dL_dinput = dL_dinput.reshape(self.input_data.shape)

        # Update the weights and biases based on the learning rate and gradients
        self.weights -= lr * dL_dw
        self.biases -= lr * dL_db

        # Return the gradient of the loss with respect to the input data
        return dL_dinput

    

In [4]:
X_train = train_images[:50] / 255.0
y_train = train_labels[:50]

X_test = train_images[5:10] / 255.0
y_test = train_labels[5:10]

X_train[0].shape

(28, 28)

In [5]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [6]:
13 * 13 * 6

1014

In [7]:
conv = Convolution(X_train[0].shape, 4, 6)
pool = MaxPool(2)
full = Fully_Connected(864,10)

def cross_entropy_loss(predictions, targets):
    
    num_samples = 10

    # Avoid numerical instability by adding a small epsilon value
    epsilon = 1e-7
    predictions = np.clip(predictions, epsilon, 1 - epsilon)

    # Calculate the categorical cross-entropy loss
    loss = -np.sum(targets * np.log(predictions)) / num_samples

    return loss

def cross_entropy_loss_gradient(actual_labels, predicted_probs):
    num_samples = actual_labels.shape[0]

    # Calculate the gradient of the cross-entropy loss function
    gradient = -actual_labels / (predicted_probs + 1e-7) / num_samples

    return gradient


def forward(input_data, label):
    layer_out = input_data
    loss = 0
    for cnn_forward in cnn:
        layer_out = cnn_forward.forward(layer_out)
    #acc = 1 if any(np.argmax(layer_out)) == label else 0
    loss = categorical_cross_entropy_loss(layer_out, label)
    
    return layer_out

def train_network(X, y, conv, pool, full, lr=0.01, epochs=100):
    for epoch in range(epochs):
        total_loss = 0.0
        correct_predictions = 0

        for i in range(len(X)):
            # Forward pass
            conv_out = conv.forward(X[i])
            pool_out = pool.forward(conv_out)
            full_out = full.forward(pool_out)

            # Calculate loss and accuracy
            loss = cross_entropy_loss(full_out.flatten(), y[i])
            total_loss += loss
            
            # Backward pass
            gradient = cross_entropy_loss_gradient(y[i], full_out.flatten()).reshape((-1, 1))
            full_back = full.backward(gradient, lr)
            pool_back = pool.backward(full_back, lr)
            conv_back = conv.backward(pool_back, lr)

            # Print the values for debugging
            #print(f"Epoch: {epoch+1}, Sample: {i+1}")
            #print("Convolution output shape:", conv_out.shape)
            #print("Pooling output shape:", pool_out.shape)
            #print("Fully Connected output shape:", full_out.shape)
            #print("Gradient shape:", gradient.shape)
            #print("Fully Connected backward shape:", full_back.shape)
            #print("Pooling backward shape:", pool_back.shape)
            #print("Convolution backward shape:", conv_back.shape)
            #print()

        # Print epoch statistics
        average_loss = total_loss / len(X)
        accuracy = 0
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {average_loss:.4f}")
        

def predict(input_sample, conv, pool, full):
    # Forward pass through convolutional and pooling layers
    conv_out = conv.forward(input_sample)
    pool_out = pool.forward(conv_out)

    # Flatten the output feature maps
    flattened_output = pool_out.flatten()

    # Forward pass through fully connected layer
    predictions = full.forward(flattened_output)

    return predictions

In [8]:
train_network(X_train, y_train, conv, pool, full)

Epoch 1/100 - Loss: 1.4868
Epoch 2/100 - Loss: 1.4751
Epoch 3/100 - Loss: 1.4595
Epoch 4/100 - Loss: 1.4293
Epoch 5/100 - Loss: 1.4064
Epoch 6/100 - Loss: 1.3719
Epoch 7/100 - Loss: 1.3592
Epoch 8/100 - Loss: 1.3478
Epoch 9/100 - Loss: 1.3377
Epoch 10/100 - Loss: 1.3320
Epoch 11/100 - Loss: 1.3263
Epoch 12/100 - Loss: 1.3208
Epoch 13/100 - Loss: 1.3135
Epoch 14/100 - Loss: 1.2884
Epoch 15/100 - Loss: 1.2668
Epoch 16/100 - Loss: 1.2541
Epoch 17/100 - Loss: 1.2356
Epoch 18/100 - Loss: 1.2284
Epoch 19/100 - Loss: 1.2036
Epoch 20/100 - Loss: 1.2237
Epoch 21/100 - Loss: 1.2057
Epoch 22/100 - Loss: 1.1929
Epoch 23/100 - Loss: 1.1683
Epoch 24/100 - Loss: 1.1470
Epoch 25/100 - Loss: 1.1258
Epoch 26/100 - Loss: 1.1164
Epoch 27/100 - Loss: 1.1123
Epoch 28/100 - Loss: 1.1091
Epoch 29/100 - Loss: 1.1062
Epoch 30/100 - Loss: 1.1022
Epoch 31/100 - Loss: 1.1009
Epoch 32/100 - Loss: 1.0995
Epoch 33/100 - Loss: 1.0985
Epoch 34/100 - Loss: 1.0980
Epoch 35/100 - Loss: 1.0976
Epoch 36/100 - Loss: 1.0974
E

In [9]:
predictions = []
for data in X_train:
    pred = predict(data, conv, pool, full)
    one_hot_pred = np.zeros_like(pred)
    one_hot_pred[np.argmax(pred)] = 1
    predictions.append(one_hot_pred)

predictions = np.array(predictions)


In [10]:
conv_out = conv.forward(X_train[0])
pool_out = pool.forward(conv_out)
full_out = full.forward(pool_out)
gradient = cross_entropy_loss_gradient(y_train[0], full_out.flatten()).reshape((-1, 1))
full_back = full.backward(gradient, 0.01)
pool_back = pool.backward(full_back, 0.01)
conv_back = conv.backward(pool_back, 0.01)

In [11]:
13 * 13 * 10

1690